<a href="https://colab.research.google.com/github/rishis123/HealthTrendsCode/blob/main/WastewaterAndHospitalization_HealthTrends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Per the meeting on 6/17/2024 -- I will be trying to connect the Wastewater (https://data.cdc.gov/Public-Health-Surveillance/NWSS-Public-SARS-CoV-2-Wastewater-Metric-Data/2ew6-ywp6/about_data) and Hospitalization datasets (https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/anag-cw7u/about_data)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip3 install vaex


In [ ]:

# Path to the CSV file
file_path = 'https://data.cdc.gov/api/views/2ew6-ywp6/rows.csv?accessType=DOWNLOAD'

# Specify data types for problematic columns
dtype = {
    'county_fips': 'str',  #to account for Missouri 190 -- multiple codes
}

wastewater_df = pd.read_csv(file_path,dtype=dtype)

print(wastewater_df)


       wwtp_jurisdiction  wwtp_id reporting_jurisdiction  sample_location  \
0         South Carolina     2564         South Carolina  Treatment plant   
1         South Carolina     2564         South Carolina  Treatment plant   
2         South Carolina     2564         South Carolina  Treatment plant   
3         South Carolina     2564         South Carolina  Treatment plant   
4         South Carolina     2564         South Carolina  Treatment plant   
...                  ...      ...                    ...              ...   
770985             Texas     2326                  Texas  Treatment plant   
770986             Texas     2326                  Texas  Treatment plant   
770987             Texas     2326                  Texas  Treatment plant   
770988             Texas     2326                  Texas  Treatment plant   
770989             Texas     2326                  Texas  Treatment plant   

        sample_location_specify  \
0                           NaN   
1    

In [ ]:
print(wastewater_df.dtypes)

wwtp_jurisdiction           object
wwtp_id                      int64
reporting_jurisdiction      object
sample_location             object
sample_location_specify    float64
key_plot_id                 object
county_names                object
county_fips                 object
population_served            int64
date_start                  object
date_end                    object
ptc_15d                    float64
detect_prop_15d            float64
percentile                 float64
sampling_prior              object
first_sample_date           object
dtype: object


From the CDC Website:

wwtp_jurisdiction (string):
- State or territory where wastewater plant is located

wwtp_id (int 64):
- Unique identifier for wastewater treatment plants

reporting_jurisdiction (string):
- CDC jurisdiction reporting this data (IRRELEVANT)

sample_location (string):
- If sample at wastewater treatment plant or upstream (IRRELEVANT)

sample_location_specify (float64):
- Unique identifier for previous (IRRELEVANT)

key_plot_id (string):
- Identifier for sewershed (IRRELEVANT)

county_names (string):
- Name for county serviced

county_fips (string):
- Code for county serviced

population_served (int64):
- number of people serviced

date_start (string):
date_end (string):
- Year-Month-Date format

ptc_15d (float64):
- Percent change in COVID Levels from previous week

detect_prop_15d (float64):
- Proportion of tests with COVID detected

percentile (float64):
- Comparison to historical levels at same site. 0% is smaller than ever, 100% is greater than ever.

sampling_prior (string):
- Sampling before December 2021 (IRRELEVANT)


first_sample_date (string):
- First date of sampling (IRRELEVANT)








First, I will drop all the irrelevant columns (likely no predictive value).

In [ ]:
# Define the columns to drop
columns_to_drop = ['reporting_jurisdiction', 'county_names', 'wwtp_id', 'wwtp_jurisdiction', 'population_served', 'sample_location', 'sample_location_specify', 'key_plot_id', 'sampling_prior', 'first_sample_date']

# Ensure only existing columns are dropped
columns_to_drop = [col for col in columns_to_drop if col in wastewater_df.columns]

wastewater_df = wastewater_df.rename(columns={"county_fips": "fips"})

wastewater_df = wastewater_df[(wastewater_df['ptc_15d'] >= -100) & (wastewater_df['ptc_15d'] <= 100)]

# Drop the specified columns
wastewater = wastewater_df.drop(columns=columns_to_drop)


# Check for NaN values in specific columns
columns_to_check = ['ptc_15d', 'detect_prop_15d', 'percentile']
wastewater = wastewater.dropna(subset=columns_to_check)

print(wastewater.head(50))

     fips  date_start    date_end  ptc_15d  detect_prop_15d  percentile
2   45051  2023-12-21  2024-01-04    -97.0            100.0      85.500
3   45051  2023-12-22  2024-01-05    -97.0            100.0      85.500
4   45051  2023-12-23  2024-01-06    -97.0            100.0      85.500
5   45051  2023-12-24  2024-01-07    -97.0            100.0      85.500
6   45051  2023-12-25  2024-01-08   -100.0            100.0      67.333
7   45051  2023-12-26  2024-01-09   -100.0            100.0      67.333
8   45051  2023-12-27  2024-01-10   -100.0            100.0      57.750
9   45051  2023-12-28  2024-01-11   -100.0            100.0      57.750
10  45051  2023-12-29  2024-01-12   -100.0            100.0      57.750
11  45051  2023-12-30  2024-01-13   -100.0            100.0      57.750
12  45051  2023-12-31  2024-01-14   -100.0            100.0      57.750
13  45051  2024-01-01  2024-01-15   -100.0            100.0      57.750
14  45051  2024-01-02  2024-01-16    -92.0            100.0     

We follow the same procedure as for hospitalization below -- we make the date a monthly attribute based on the start date. Then, aggregating on fips and start date, we take the average of ptc_15d, detect_prop_15d, and percentile.

In [ ]:
#Changing from Y-M-D to Y-M-01 (note we are going up to monthly for the sake of clarity, and ease when considering overlapping time intervals (different start dates for collection week))
wastewater['date_start'] = pd.to_datetime(wastewater['date_start'], format='%Y-%m-%d', errors='coerce').dt.strftime('%Y-%m-01')

wastewater = wastewater.drop(columns =[ 'date_end'])

#Convert to ds -- later merge with wastewater data on this column and fips.
wastewater = wastewater.rename(columns={"date_start": "ds"})

In [ ]:
# Group by fips and ds, then calculate the average of ptc_15d, detect_prop_15d, and percentile
average_values = wastewater.groupby(['fips', 'ds']).agg({
    'ptc_15d': 'mean',
    'detect_prop_15d': 'mean',
    'percentile': 'mean'
}).reset_index()

# Rename the aggregated columns
average_values = average_values.rename(columns={
    'ptc_15d': 'percent_change_month',
    'detect_prop_15d': 'detect_prop_month',
    'percentile': 'percentile_month'
})

# Merge the averages back into the wastewater DataFrame
wastewater = pd.merge(wastewater, average_values, on=['fips', 'ds'], how='left')

# Drop the original columns ptc_15d, detect_prop_15d, and percentile
wastewater = wastewater.drop(columns=['ptc_15d', 'detect_prop_15d', 'percentile'])


In [ ]:

# Drop duplicates based on 'fips' and 'ds' to keep only one row per average
wastewater = wastewater.drop_duplicates(subset=['fips', 'ds'])


wastewater.head(20) #this is the final wastewater graph


,fips,ds,percent_change_month,detect_prop_month,percentile_month
0,45051,2023-12-01,-62.352941,100.000000,68.812706
11,45051,2024-01-01,-17.600000,100.000000,80.407200
22,45051,2024-02-01,-41.270270,100.000000,85.245919
42,45051,2024-03-01,-36.325000,85.425000,41.978725
66,45051,2024-04-01,-54.594595,45.675676,19.322486
86,45051,2024-05-01,-26.529412,87.735294,26.071529
106,08039,2023-07-01,-74.052632,49.421053,43.372842
125,08039,2023-08-01,-40.619048,95.095238,58.028571
146,08039,2023-09-01,-92.937500,58.437500,57.515625
162,08039,2023-10-01,-79.272727,89.090909,65.831818


Let's also get the COVID hospitalization data, using the same strategy.

In [ ]:
  # Path to the CSV file
file_path = '/content/3_hospital_only_inpatient_beds_occupied_no_empty_rows - Copy.csv'

# Define the columns to check for values below 0
columns_to_check = [
    'all_adult_hospital_inpatient_bed_occupied_7_day_avg'
]

# Columns to read from the CSV file
#hospital code is not really useful if taking average across fips-code for a given week
columns_to_read = ['collection_week', 'fips_code',
                   'all_adult_hospital_inpatient_bed_occupied_7_day_avg']

# Read the specified columns from the CSV file
hospital = pd.read_csv(file_path, dtype=str, usecols=columns_to_read)

# Convert the specified columns to numeric, forcing errors to NaN
for column in columns_to_check:
    hospital[column] = pd.to_numeric(hospital[column], errors='coerce')

# Drop rows where any of the specified columns contain values below 0
for column in columns_to_check:
    hospital = hospital[hospital[column] >= 0] #note: around first 1,000 pages of query yields -999,999 (unclear meaning -- drop rows entirely)

# Rename the specified column
hospital = hospital.rename(columns={"all_adult_hospital_inpatient_bed_occupied_7_day_avg": "occupied_beds"})

In [ ]:
#Changing from Y/M/D to Y-M-01 (note we are going up to monthly for the sake of clarity, and ease when considering overlapping time intervals (different start dates for collection week))
hospital['collection_week'] = pd.to_datetime(hospital['collection_week'], format='%Y/%m/%d', errors='coerce').dt.strftime('%Y-%m-01')


#Convert to ds -- later merge with wastewater data on this column and fips.
hospital = hospital.rename(columns={"collection_week": "ds", "fips_code": 'fips'})

hospital.head(50)

,ds,fips,occupied_beds
0,2020-07-01,13207,17.5
1,2020-09-01,15003,20.8
2,2020-09-01,20013,8.1
3,2020-09-01,22003,7.0
4,2020-08-01,31013,8.2
5,2020-08-01,34025,167.9
6,2020-08-01,39113,35.1
8,2020-07-01,49013,14.5
9,2023-10-01,50001,26.7
10,2020-07-01,55111,26.3


Now, we start the actual calculations and final polishing of this table -- we have gone from weekly collection data to the first date of the corresponding month. The cost in period specificity came at the advantage of ease, given that different records put different start dates for the week.

Now, to get the data to a simple time series form with the y-variable being occupied bed count (effectively, average hospitalization), we take the average of the measure for every hospital in a county for that month (that is, we aggregate by ds and fips_code, and take the average of every occupied_beds count, and then multiply by 4.34524 to account for the number of weeks in a month). The end result is data that simply has the average hospitalization count across each month for the entire county, which is convenient for comparisons with the wastewater data.

In [ ]:
# Group by fips_code and collection_week, then calculate the average occupied_beds
average_beds_per_week = hospital.groupby(['fips', 'ds'])['occupied_beds'].mean().reset_index()

# Multiply the average by 4.34524 to account for average per month (4.34525 weeks in a month)
average_beds_per_week['hospitalized_monthly'] = average_beds_per_week['occupied_beds'] * 4.34524

# Merge the adjusted averages back into the original DataFrame and rename the column
hospital = pd.merge(hospital, average_beds_per_week[['fips', 'ds', 'hospitalized_monthly']], on=['fips', 'ds'])
hospital = hospital.drop(columns = ['occupied_beds'])



In [ ]:
#Ignore occupied_beds, we have monthly hospitalization data per county
hospital = hospital.drop_duplicates(subset=['fips', 'ds'])
hospital = hospital.sort_values(by='ds')
store_hospital = hospital
#hospital.shape - (106490, 3)

In [ ]:
# Sort DataFrame by 'ds' column in chronological order
wastewater = wastewater.sort_values(by='ds')

wastewater.head(50)
#wastewater.shape - (14475, 5)

,fips,ds,percent_change_month,detect_prop_month,percentile_month
290388,26041,2021-11-01,-7.500000,100.0,83.333500
106183,41067,2021-11-01,-23.885714,100.0,27.823829
100604,"29037,29095",2021-11-01,31.777778,100.0,92.500000
394164,26077,2021-11-01,2.250000,100.0,79.291625
213816,29043,2021-11-01,-25.000000,100.0,95.000000
325533,41041,2021-11-01,-46.615385,100.0,39.256423
445895,29123,2021-11-01,-58.857143,100.0,80.142857
240348,"29101,20209,29095",2021-11-01,-2.444444,100.0,96.666667
445168,26025,2021-11-01,-3.200000,100.0,93.700000
121878,54055,2021-11-01,-34.153846,100.0,46.528769


We further try to resolve the problem of messy data by deleting fips codes that are only in one dataset or another, as I want to initially try predicting future hospitalization data with past data, and then try predicting future hospitalization data with past wastewater data (which would require shared counties, as I'm predicting on a county-by-county basis).

In [ ]:
common_fips = set(wastewater['fips']).intersection(set(hospital['fips']))

# Filter dataframes to keep only common fips
wastewater_filtered = wastewater[wastewater['fips'].isin(common_fips)]
hospital_filtered = hospital[hospital['fips'].isin(common_fips)]

#wastewater.shape - (14475, 5)
#wastewater_filtered.shape - (12150, 5)

#hospital.shape - (106490, 3)
#hospital_filtered.shape - (29902, 3) NOTE SIGNIFICANT REDUCTION


Finally, we can begin the machine learning models. Consider ARIMA.

The ARMA/ARIMA suite excel in outlier/seasonality handling. In the choice between the two, the ARIMA is better suited for non-stationary data (with a changing mean over time) -- we determine below whether this is the case.

https://www.geeksforgeeks.org/how-to-check-if-time-series-data-is-stationary-with-python/#.
The method is described further here: we will set an alpha-level of 0.05, meaning p>0.05 will fail to reject the null hypothesis of non-stationary data, else p<=0.05 will support the alternate hypotehsis of stationary data.

In [ ]:
# Function to sample a FIPS code ensuring it has more than 30 rows
def sample_fips_with_min_rows(common_fips, min_rows=10):
    while True:
        sampled_fips = np.random.choice(list(common_fips), size=3, replace=False)
        # Check if each sampled FIPS code has more than 10 rows in hospital_filtered
        if all((hospital_filtered['fips'] == fips).sum() > min_rows for fips in sampled_fips):
            # Check if each sampled FIPS code has more than 10 rows in wastewater_filtered
            if all((wastewater_filtered['fips'] == fips).sum() > min_rows for fips in sampled_fips):
                return sampled_fips
#Note -- the wastewater_filtered is for later on, rather than computing the average (lots of missing rows - data values)

# Sample 3 FIPS codes ensuring each has more than 10 rows
sampled_fips = sample_fips_with_min_rows(common_fips, min_rows=30)

# Select data for the first sampled FIPS code
df_1 = hospital_filtered[hospital_filtered['fips'] == sampled_fips[0]]
hospital_1 = df_1[['ds', 'hospitalized_monthly', 'fips']].copy()
hospital_1 = hospital_1.rename(columns={"hospitalized_monthly": "y"})

# Select data for the second sampled FIPS code
df_2 = hospital_filtered[hospital_filtered['fips'] == sampled_fips[1]]
hospital_2 = df_2[['ds', 'hospitalized_monthly', 'fips']].copy()
hospital_2 = hospital_2.rename(columns={"hospitalized_monthly": "y"})

# Select data for the third sampled FIPS code
df_3 = hospital_filtered[hospital_filtered['fips'] == sampled_fips[2]]
hospital_3 = df_3[['ds', 'hospitalized_monthly', 'fips']].copy()
hospital_3 = hospital_3.rename(columns={"hospitalized_monthly": "y"})

print(hospital_1)
print(hospital_2)
print(hospital_3)

list_of_hospitals = [hospital_1, hospital_2, hospital_3]  # List of DataFrames


                ds           y   fips
499204  2020-07-01   64.526814  26139
503660  2020-08-01  156.631418  26139
490426  2020-09-01  158.021895  26139
508261  2020-10-01  165.553644  26139
784098  2020-11-01  205.935408  26139
782290  2020-12-01  181.087877  26139
742484  2021-01-01  145.855223  26139
629494  2021-02-01  142.234189  26139
202315  2021-03-01  166.350271  26139
487351  2021-04-01  182.789763  26139
208957  2021-05-01  175.634601  26139
744550  2021-06-01  162.475766  26139
513497  2021-07-01  163.163762  26139
484234  2021-08-01  169.174677  26139
687522  2021-09-01  188.692047  26139
621931  2021-10-01  197.042150  26139
621768  2021-11-01  226.749107  26139
692956  2021-12-01  210.164775  26139
160675  2022-01-01  181.109603  26139
208049  2022-02-01  169.536781  26139
509461  2022-03-01  156.175168  26139
271568  2022-04-01  153.893917  26139
181374  2022-05-01  154.603639  26139
167711  2022-06-01  154.509492  26139
497660  2022-07-01  170.043725  26139
630727  2022

In [ ]:
# from statsmodels.tsa.stattools import adfuller


# for hospital in list_of_hospitals:
#   # Extracting the 'y' column which contains the time series values
#   values = hospital['y']

#   # Passing the extracted values to the adfuller function
#   res = adfuller(values)


#   # Printing the statistical result of the adfuller test
#   print('Augmented Dickey_fuller Statistic: %f' % res[0])
#   print('p-value: %f' % res[1])

#   # printing the critical values at different alpha levels.
#   print('critical values at different levels:')
#   for k, v in res[4].items():
#       print('\t%s: %.3f' % (k, v))

Note all p-values are below 0.05. As such, we assume the data is not stationary.


https://www.kaggle.com/code/iamleonie/time-series-interpreting-acf-and-pacf. This explains the next steps -- checking ACF vs PACF, to further determine if both the AR and MA models (individual compoennts of the ARIMA model) are suited to the dataset. ACF differs from PACF as it includes intermediary lag/period's effects on the period of prediction, unlike the PACF coefficient for just a single lag.

First, however, we take the first difference to make the data stationary.

In [ ]:
# for hospital in list_of_hospitals:
#   hospital['y_diff'] = hospital['y'].diff().fillna(0)

#   # Extracting the 'y' column which contains the time series values
#   values = hospital['y_diff']

#   # Passing the extracted values to the adfuller function
#   res = adfuller(values)


#   # Printing the statistical result of the adfuller test
#   print('Augmented Dickey_fuller Statistic: %f' % res[0])
#   print('p-value: %f' % res[1])

#   # printing the critical values at different alpha levels.
#   print('critical values at different levels:')
#   for k, v in res[4].items():
#       print('\t%s: %.3f' % (k, v))


All the data is indeed stationary, with consistent 0.000 p-values.

In [ ]:
# for hospital in list_of_hospitals:
#   f, ax = plt.subplots(nrows=2, ncols=1, figsize=(12, 8))

#   # Plot the original time series
#   sns.lineplot(x=hospital['ds'], y=hospital['y'], marker='o', ax=ax[0])
#   ax[0].set_xlim([hospital['ds'].iloc[0], hospital['ds'].iloc[-1]])
#   ax[0].set_title('Hospital Time Series')

#   # Plot the differenced time series
#   sns.lineplot(x=hospital['ds'], y=hospital['y_diff'], marker='o', ax=ax[1])
#   ax[1].set_xlim([hospital['ds'].iloc[0], hospital['ds'].iloc[-1]])
#   ax[1].set_title('Differenced Hospital Time Series')

#   plt.tight_layout()
#   plt.show()


The differenced time series look like they could either be random noise, or potentially a trend over time. We verify with acf and pacf.

In [ ]:
# from pandas.plotting import register_matplotlib_converters
# from statsmodels.tsa.stattools import acf, pacf
# from statsmodels.tsa.arima_model import ARMA
# register_matplotlib_converters()
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from time import time

In [ ]:
# for hospital in list_of_hospitals:
#   f, ax = plt.subplots(nrows=2, ncols=1, figsize=(12, 8))
#   plot_acf(hospital['y_diff'],lags=15, ax=ax[0])
#   plot_pacf(hospital['y_diff'],lags=15, ax=ax[1], method='ols')

#   plt.tight_layout()
#   plt.show()


It appears the data is white noise, and cannot be predicted by ARIMA, but certain counties have annual cycles. We try aggregating the data, regardless of county (so it's just a national prediction of hospitalization) -- and seeing if the ACF and PACF is different.

In [ ]:
# # Group by 'ds' and sum the 'hospitalized_monthly' counts for each month
# hospital_national = hospital_filtered.groupby('ds').agg({'hospitalized_monthly': 'sum'}).reset_index()

# # Display the first 50 rows of the aggregated DataFrame
# print(hospital_national.head(50))

In [ ]:
# # Plot the aggregated national hospitalization data
# plt.figure(figsize=(12, 6))
# sns.lineplot(x=hospital_national['ds'], y=hospital_national['hospitalized_monthly'], marker='o')
# plt.title('National Hospitalization Time Series')
# plt.xlabel('Date')
# plt.ylabel('Hospitalization Count')
# plt.show()

The data appears vaguely cyclic from August 2020 - April 2024. We proceed with ARIMA as a whole for just this interval.

In [ ]:
# # Filter out rows before August 2020
# start_date = '2020-08-01'
# hospital_national = hospital_national[hospital_national['ds'] >= start_date]
# print(hospital_national)

In [ ]:
# #Check for stationary:
# values = hospital_national['hospitalized_monthly']

#   # Passing the extracted values to the adfuller function
# res = adfuller(values)


#   # Printing the statistical result of the adfuller test
# print('Augmented Dickey_fuller Statistic: %f' % res[0])
# print('p-value: %f' % res[1])

#   # printing the critical values at different alpha levels.
# print('critical values at different levels:')
# for k, v in res[4].items():
#   print('\t%s: %.3f' % (k, v))

#   #Data is stationary -- we proceed with ARMA without any differences.


In [ ]:
# f, ax = plt.subplots(nrows=2, ncols=1, figsize=(12, 8))
# plot_acf(hospital_national['hospitalized_monthly'],lags=15, ax=ax[0])
# plot_pacf(hospital_national['hospitalized_monthly'],lags=15, ax=ax[1], method='ols')

# plt.tight_layout()
# plt.show()

ARMA/ARIMA is also not a good fit for the national data -- we return to county-level data, and look for the poisson regression as before. The requirement is mean ~= variance.

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.diagnostic import het_breuschpagan

for hospital in list_of_hospitals:
  check_mean_variance = hospital[['y']]

  mean_absent = check_mean_variance.mean()
  var_absent = check_mean_variance.var()
  print(f'Mean: {mean_absent}, Variance: {var_absent}')

Mean: y    162.670199
dtype: float64, Variance: y    546.233198
dtype: float64
Mean: y    58.461493
dtype: float64, Variance: y    160.57166
dtype: float64
Mean: y    41.819906
dtype: float64, Variance: y    40.87037
dtype: float64


The variance and mean are very different, so the data is overdispersed. We try the negative binomial regression.https://timeseriesreasoning.com/contents/negative-binomial-regression-model/

In [ ]:
from patsy import dmatrices
import statsmodels.api as sm
import statsmodels.formula.api as smf


# Merge hospital_df_1 with wastewater_df
merged_df_1 = pd.merge(list_of_hospitals[0], wastewater_filtered, on=['ds', 'fips'], how='inner')

# Merge hospital_df_2 with wastewater_df
merged_df_2 = pd.merge(list_of_hospitals[1], wastewater_filtered, on=['ds', 'fips'], how='inner')

# Merge hospital_df_3 with wastewater_df
merged_df_3 = pd.merge(list_of_hospitals[2], wastewater_filtered, on=['ds', 'fips'], how='inner')

# Create predict_1 DataFrame with replaced missing wastewater data
predict_1 = merged_df_1[['ds', 'fips', 'y', 'percent_change_month', 'detect_prop_month', 'percentile_month']]
print(predict_1)

# Create predict_2 DataFrame
predict_2 = merged_df_2[['ds', 'fips', 'y', 'percent_change_month', 'detect_prop_month', 'percentile_month']]
print(predict_2)

# Create predict_3 DataFrame
predict_3 = merged_df_3[['ds', 'fips', 'y', 'percent_change_month', 'detect_prop_month', 'percentile_month']]
print(predict_3)
to_predict = [predict_1, predict_2, predict_3]


            ds   fips           y  percent_change_month  detect_prop_month  \
0   2021-11-01  26139  226.749107            -31.433333         100.000000   
1   2021-12-01  26139  210.164775             -0.983333          96.000000   
2   2022-01-01  26139  181.109603            -45.510870          96.684783   
3   2022-02-01  26139  169.536781            -30.974684          28.822785   
4   2022-03-01  26139  156.175168            -11.358974          11.957265   
5   2022-04-01  26139  153.893917              7.988439          60.115607   
6   2022-05-01  26139  154.603639             -3.411321          94.773585   
7   2022-06-01  26139  154.509492            -11.871111          89.955556   
8   2022-07-01  26139  170.043725             -3.156627          93.353414   
9   2022-08-01  26139  165.481223            -21.279635          97.218845   
10  2022-09-01  26139  155.704433            -18.893064          88.497110   
11  2022-10-01  26139  164.684596            -10.596439         

'percent_change_month', 'detect_prop_month', 'percentile_month' are all the additional regressors we will be using -- they are from the wastewater dataframe.

In [ ]:
import pandas as pd
import numpy as np

from patsy import dmatrices

import statsmodels.api as sm
from statsmodels.formula.api import glm

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

In [ ]:

# Extracting month and year for categorical treatment
to_predict[0]['ds'] = pd.to_datetime(to_predict[0]['ds'])
to_predict[0]['month'] = to_predict[0]['ds'].dt.strftime('%m')  # Month as a string
to_predict[0]['year'] = to_predict[0]['ds'].dt.year  # Year as a numeric variable

to_predict[1]['ds'] = pd.to_datetime(to_predict[1]['ds'])
to_predict[1]['month'] = to_predict[1]['ds'].dt.strftime('%m')  # Month as a string
to_predict[1]['year'] = to_predict[1]['ds'].dt.year  # Year as a numeric variable

to_predict[2]['ds'] = pd.to_datetime(to_predict[2]['ds'])
to_predict[2]['month'] = to_predict[2]['ds'].dt.strftime('%m')  # Month as a string
to_predict[2]['year'] = to_predict[2]['ds'].dt.year  # Year as a numeric variable



In [ ]:
#Split to train and test sets (80% of observations in train set)

#for all 3 dataframes (each a fips code) in the list -- don't use for loop to distinguish vars
mask = np.random.rand(len(to_predict[0])) < 0.8
df_0_train = to_predict[0][mask]
df_0_test = to_predict[0][~mask]

mask = np.random.rand(len(to_predict[1])) < 0.8
df_1_train = to_predict[1][mask]
df_1_test = to_predict[1][~mask]

mask = np.random.rand(len(to_predict[2])) < 0.8
df_2_train = to_predict[2][mask]
df_2_test = to_predict[2][~mask]


In [ ]:
print(df_0_train)
print(df_0_test)

           ds   fips           y  percent_change_month  detect_prop_month  \
0  2021-11-01  26139  226.749107            -31.433333         100.000000   
1  2021-12-01  26139  210.164775             -0.983333          96.000000   
2  2022-01-01  26139  181.109603            -45.510870          96.684783   
3  2022-02-01  26139  169.536781            -30.974684          28.822785   
6  2022-05-01  26139  154.603639             -3.411321          94.773585   
7  2022-06-01  26139  154.509492            -11.871111          89.955556   
9  2022-08-01  26139  165.481223            -21.279635          97.218845   
10 2022-09-01  26139  155.704433            -18.893064          88.497110   
13 2022-12-01  26139  176.380534            -23.746032          87.440476   
15 2023-02-01  26139  157.659791            -21.738516          69.494700   
16 2023-03-01  26139  151.214352            -17.622024          71.241071   
17 2023-04-01  26139  148.549271            -15.194986          33.894150   

In [ ]:
print(df_1_train)
print(df_1_test)

           ds   fips          y  percent_change_month  detect_prop_month  \
0  2021-11-01  53021  50.622046             37.833333           0.000000   
1  2021-12-01  53021  51.925618            -10.333333          15.533333   
2  2022-01-01  53021  63.266694            -36.178571          89.785714   
3  2022-02-01  53021  55.619072            -16.500000           4.642857   
4  2022-03-01  53021  38.020850            -41.120000          11.200000   
5  2022-04-01  53021  43.235138             26.785714         100.000000   
6  2022-05-01  53021  44.929782             51.913043         100.000000   
7  2022-06-01  53021  49.427105             11.954545         100.000000   
8  2022-07-01  53021  48.405974              3.870968         100.000000   
9  2022-08-01  53021  47.145854            -20.166667         100.000000   
10 2022-09-01  53021  48.449426            -31.730769         100.000000   
11 2022-10-01  53021  42.496447            -16.421053         100.000000   
12 2022-11-0

In [ ]:
print(df_2_train)
print(df_2_test)

           ds   fips          y  percent_change_month  detect_prop_month  \
1  2021-12-01  29009  49.225362            -70.000000         100.000000   
2  2022-01-01  29009  46.624425            -71.909091         100.000000   
3  2022-02-01  29009  44.031765            -89.058824         100.000000   
4  2022-03-01  29009  51.273832            -48.791667          49.291667   
6  2022-05-01  29009  37.224223            -83.150000          80.850000   
7  2022-06-01  29009  46.856171            -62.937500         100.000000   
8  2022-07-01  29009  37.707027            -27.421053         100.000000   
9  2022-08-01  29009  37.206118             -4.724138         100.000000   
10 2022-09-01  29009  46.841687            -28.045455         100.000000   
11 2022-10-01  29009  43.235138            -61.700000          95.000000   
12 2022-11-01  29009  42.335053            -72.578947          84.210526   
13 2022-12-01  29009  47.870061             12.000000         100.000000   
14 2023-01-0

In [ ]:
# Variables - patsy notation

#dependent var is y, regressors on other side of ~


expr = """ y ~ C(month) + year + percent_change_month + detect_prop_month + percentile_month """

In [ ]:
y_0_train, X_0_train = dmatrices(expr, df_0_train, return_type = 'dataframe')

y_0_test, X_0_test = dmatrices(expr, df_0_test, return_type = 'dataframe')

y_1_train, X_1_train = dmatrices(expr, df_1_train, return_type = 'dataframe')

y_1_test, X_1_test = dmatrices(expr, df_1_test, return_type = 'dataframe')

y_2_train, X_2_train = dmatrices(expr, df_2_train, return_type = 'dataframe')

y_2_test, X_2_test = dmatrices(expr, df_2_test, return_type = 'dataframe')


print("X_0_train shape:", X_0_train)
print("X_0_test shape:", X_0_test)

#result is design matrix -- organizes data into matrix for model fitting

X_0_train shape:     Intercept  C(month)[T.02]  C(month)[T.03]  C(month)[T.04]  C(month)[T.05]  \
0         1.0             0.0             0.0             0.0             0.0   
1         1.0             0.0             0.0             0.0             0.0   
2         1.0             0.0             0.0             0.0             0.0   
3         1.0             1.0             0.0             0.0             0.0   
6         1.0             0.0             0.0             0.0             1.0   
7         1.0             0.0             0.0             0.0             0.0   
9         1.0             0.0             0.0             0.0             0.0   
10        1.0             0.0             0.0             0.0             0.0   
13        1.0             0.0             0.0             0.0             0.0   
15        1.0             1.0             0.0             0.0             0.0   
16        1.0             0.0             1.0             0.0             0.0   
17        1

 "For example, in cases where the response variable is expected to be always positive and varying over a wide range, constant input changes lead to geometrically (i.e. exponentially) varying, rather than constantly varying, output changes." - From Wikipedia, reason for Generalized Linear Regression, as opposed to poisson (overdispersion). Codebase - https://gist.github.com/farid-mammadaliyev/775588d371316b25de3b62186b5898ec

Additional clarification: The mean of the negative binomial distribution is μ. The variance is  μ + α*μ^2. We specify alpha = 1, so that the variance becomes  μ + μ^2, meaning the variance increases quadratically with the mean -- dealing with the overdispersion.

In [ ]:
# Regression

nb_training_results_0 = sm.GLM(y_0_train, X_0_train, family = sm.families.NegativeBinomial(alpha = 1)).fit()
#nb_training_results holds the fitted model results, including estimated coefficients, standard errors, p-values, and various metrics for evaluating model fit.

print(nb_training_results_0.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   20
Model:                            GLM   Df Residuals:                        5
Model Family:        NegativeBinomial   Df Model:                           14
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -121.95
Date:                Mon, 24 Jun 2024   Deviance:                     0.018947
Time:                        17:16:51   Pearson chi2:                   0.0190
No. Iterations:                     4   Pseudo R-squ. (CS):            0.01339
Covariance Type:            nonrobust                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept              187.5806 

In [ ]:
# Regression

nb_training_results_1 = sm.GLM(y_1_train, X_1_train, family = sm.families.NegativeBinomial(alpha = 1)).fit()

print(nb_training_results_1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   26
Model:                            GLM   Df Residuals:                       10
Model Family:        NegativeBinomial   Df Model:                           15
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -129.27
Date:                Mon, 24 Jun 2024   Deviance:                      0.31999
Time:                        17:16:51   Pearson chi2:                    0.321
No. Iterations:                     5   Pseudo R-squ. (CS):            0.01983
Covariance Type:            nonrobust                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             -241.9076 

In [ ]:
# Regression

nb_training_results_2 = sm.GLM(y_2_train, X_2_train, family = sm.families.NegativeBinomial(alpha = 1)).fit()

print(nb_training_results_2.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   27
Model:                            GLM   Df Residuals:                       11
Model Family:        NegativeBinomial   Df Model:                           15
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -127.92
Date:                Mon, 24 Jun 2024   Deviance:                      0.12358
Time:                        17:16:51   Pearson chi2:                    0.122
No. Iterations:                     4   Pseudo R-squ. (CS):           0.007319
Covariance Type:            nonrobust                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept              152.8312 

The most important column is P > |Z|, which is just the p-value of the result. We see that detect_prop_month is sometimes, but not always -- significant, referring to a p-value of < 0.05.

We want to use the mean column, in association with the X_train data, to find the R^2 score, showing how close the association is in between the two variables.

In [ ]:
# Generate predictions and confidence intervals for train data
nb_prediction_train_0 = nb_training_results_0.get_prediction(X_0_train)
nb_summary_frame_train_0 = nb_prediction_train_0.summary_frame()

# Print the summary frame
print(nb_summary_frame_train_0)

# Extract the predicted counts
predicted_counts_train_0 = nb_summary_frame_train_0['mean']

# Calculate and print the R-squared value for the train set
r2_0 = r2_score(y_0_train, predicted_counts_train_0)
print("R-square of train set: ", round(r2_0 * 100, 2), "%")

          mean     mean_se  mean_ci_lower  mean_ci_upper
0   226.749107  227.248557      31.803077    1616.672430
1   204.529688  196.111172      31.231337    1339.436516
2   175.371680  159.205002      29.595530    1039.184828
3   173.999288  152.600971      31.190773     970.663751
6   157.680100  135.139177      29.393925     845.855522
7   154.509492  155.008686      21.627361    1103.841695
9   157.160919  125.240631      32.963017     749.311103
10  155.704433  156.203633      21.795680    1112.324584
13  180.885576  157.053030      32.987720     991.871879
15  161.432039  108.009765      43.498477     599.108416
16  159.658421  124.602192      34.585078     737.046507
17  148.784125  113.560104      33.332928     664.109548
18  147.857594  126.737538      27.556761     793.339550
21  146.556677  116.801258      30.734321     698.855851
23  167.031026  167.530279      23.391139    1192.732142
25  161.116370  146.508746      27.108929     957.562171
26  163.481631  148.434893     

In [ ]:
# Generate predictions and confidence intervals for train data
nb_prediction_train_1 = nb_training_results_1.get_prediction(X_1_train)
nb_summary_frame_train_1 = nb_prediction_train_1.summary_frame()

# Print the summary frame
print(nb_summary_frame_train_1)

# Extract the predicted counts
predicted_counts_train_1 = nb_summary_frame_train_1['mean']

# Calculate and print the R-squared value for the train set
r2_1 = r2_score(y_1_train, predicted_counts_train_1)
print("R-square of train set: ", round(r2_1 * 100, 2), "%")

         mean    mean_se  mean_ci_lower  mean_ci_upper
0   46.325847  39.621618       8.665764     247.650895
1   52.817048  38.269446      12.765043     218.537494
2   56.493807  50.191300       9.902850     322.286021
3   53.393318  42.872368      11.066528     257.609830
4   46.304040  35.854182      10.151163     211.213636
5   42.206054  36.553361       7.729926     230.448640
6   42.895443  35.987084       8.285077     222.088334
7   41.398920  33.469666       8.488296     201.909840
8   44.292595  34.836811       9.480865     206.925635
9   49.347851  37.615318      11.077488     219.834177
10  48.449426  48.946872       6.688789     350.937475
11  51.710704  38.604919      11.970467     223.382829
12  50.997067  43.591312       9.548925     272.355345
13  52.464738  34.604880      14.402204     191.119972
14  56.345961  36.850830      15.637537     203.028608
15  52.779222  42.381180      10.938475     254.664955
16  47.062073  32.449017      12.183518     181.789756
17  49.581

In [ ]:
# Generate predictions and confidence intervals for train data
nb_prediction_train_2 = nb_training_results_2.get_prediction(X_2_train)
nb_summary_frame_train_2 = nb_prediction_train_2.summary_frame()

# Print the summary frame
print(nb_summary_frame_train_2)

# Extract the predicted counts
predicted_counts_train_2 = nb_summary_frame_train_2['mean']

# Calculate and print the R-squared value for the train set
r2_2 = r2_score(y_2_train, predicted_counts_train_2)
print("R-square of train set: ", round(r2_2 * 100, 2), "%")

         mean    mean_se  mean_ci_lower  mean_ci_upper
1   50.033290  40.998949      10.040517     249.322825
2   45.748150  37.373898       9.225405     226.861939
3   43.922030  30.580836      11.220984     171.922953
4   49.143567  42.076465       9.176401     263.184912
6   41.781784  31.064874       9.730076     179.414579
7   42.665978  33.983237       8.955791     203.263525
8   38.369985  32.298310       7.370287     199.755548
9   37.790559  32.628333       6.957556     205.262651
10  45.610528  34.318626      10.437523     199.311674
11  43.047859  35.881735       8.403273     220.523369
12  38.151431  28.100275       9.006622     161.606834
13  45.678616  33.296963      10.945566     190.628424
14  46.428218  28.743419      13.797756     156.226809
15  40.827486  24.849289      12.384439     134.595004
17  41.421439  35.608487       7.682075     223.342737
18  38.275207  28.462170       8.911441     164.394455
19  42.499185  33.850976       8.920538     202.474408
20  38.809

There appears to potentially be an overfitting problem occasionally due to the impossibly high R^2 score -- we try again with the test data to see if the generalizability is comparable.

In [ ]:
missing_columns = set(X_0_train.columns) - set(X_0_test.columns)

# Fill missing columns in X_1_test with appropriate values
for col in missing_columns:
    X_0_test[col] = 0  # Fill with NaN or appropriate values
X_0_test = X_0_test[X_0_train.columns]

print(X_0_test)
print(X_0_train)

    Intercept  C(month)[T.02]  C(month)[T.03]  C(month)[T.04]  C(month)[T.05]  \
4         1.0               0             1.0             0.0               0   
5         1.0               0             0.0             1.0               0   
8         1.0               0             0.0             0.0               0   
11        1.0               0             0.0             0.0               0   
12        1.0               0             0.0             0.0               0   
14        1.0               0             0.0             0.0               0   
19        1.0               0             0.0             0.0               0   
20        1.0               0             0.0             0.0               0   
22        1.0               0             0.0             0.0               0   
24        1.0               0             0.0             0.0               0   

    C(month)[T.06]  C(month)[T.08]  C(month)[T.09]  C(month)[T.10]  \
4              0.0               0    

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

# Get predictions for X_0_test
nb_prediction_test_0 = nb_training_results_0.get_prediction(X_0_test)

# Obtain summary frame for predictions
nb_summary_frame_test_0 = nb_prediction_test_0.summary_frame()

# Extract the predicted counts
predicted_counts_test_0 = nb_summary_frame_test_0['mean']
print(predicted_counts_test_0)
print(y_0_test)


r2 = r2_score(y_0_test, predicted_counts_test_0)
print("R-squared:", r2)

# Calculate Mean Absolute Error (MAE) for additional insight
mae = mean_absolute_error(y_0_test, predicted_counts_test_0)
print("Mean Absolute Error:", mae)

4     175.513570
5     169.153930
8     207.123106
11    187.795153
12    235.554908
14    186.220506
19    136.798044
20    190.645007
22    134.761207
24    224.411497
Name: mean, dtype: float64
             y
4   156.175168
5   153.893917
8   170.043725
11  164.684596
12  164.141441
14  154.632607
19  156.971795
20  151.388162
22  149.584887
24  152.554134
R-squared: -38.791280866327085
Mean Absolute Error: 34.39013579067993


In [ ]:
missing_columns = set(X_1_train.columns) - set(X_1_test.columns)

# Fill missing columns in X_1_test with appropriate values
for col in missing_columns:
    X_1_test[col] = 0  # Fill with NaN or appropriate values
X_1_test = X_1_test[X_1_train.columns]
print(X_1_test.shape)
print(X_1_train.shape)

(4, 16)
(26, 16)


In [ ]:
# Get predictions for X_1_test
nb_prediction_test_1 = nb_training_results_1.get_prediction(X_1_test)

# Obtain summary frame for predictions
nb_summary_frame_test_1 = nb_prediction_test_1.summary_frame()

# Extract the predicted counts
predicted_counts_test_1 = nb_summary_frame_test_1['mean']
print(predicted_counts_test_1)
print(y_1_test)


r2 = r2_score(y_1_test, predicted_counts_test_1)
print("R-squared:", r2)

# Calculate Mean Absolute Error (MAE) for additional insight
mae = mean_absolute_error(y_1_test, predicted_counts_test_1)
print("Mean Absolute Error:", mae)

22    59.827649
24    71.779559
27    80.529339
29    59.551459
Name: mean, dtype: float64
            y
22  54.532762
24  61.485146
27  64.961338
29  77.634955
R-squared: -1.5024349349514639
Mean Absolute Error: 12.310199185268026


In [ ]:

missing_columns = set(X_2_train.columns) - set(X_2_test.columns)

# Fill missing columns in X_1_test with appropriate values
for col in missing_columns:
    X_2_test[col] = 0  # Fill with NaN or appropriate values
X_2_test = X_2_test[X_2_train.columns]

print(X_2_test)

    Intercept  C(month)[T.02]  C(month)[T.03]  C(month)[T.04]  C(month)[T.05]  \
0         1.0               0               0             0.0               0   
5         1.0               0               0             1.0               0   
16        1.0               0               0             0.0               0   

    C(month)[T.06]  C(month)[T.07]  C(month)[T.08]  C(month)[T.09]  \
0                0               0               0               0   
5                0               0               0               0   
16               0               0               0               0   

    C(month)[T.10]  C(month)[T.11]  C(month)[T.12]    year  \
0                0             1.0               0  2021.0   
5                0             0.0               0  2022.0   
16               0             0.0               0  2023.0   

    percent_change_month  detect_prop_month  percentile_month  
0             -85.555556         100.000000         73.666667  
5             -59

In [ ]:
# Get predictions for X_2_test
nb_prediction_test_2 = nb_training_results_2.get_prediction(X_2_test)

# Obtain summary frame for predictions
nb_summary_frame_test_2 = nb_prediction_test_2.summary_frame()

# Extract the predicted counts
predicted_counts_test_2 = nb_summary_frame_test_2['mean']
print(predicted_counts_test_2)
print(y_2_test)


r2 = r2_score(y_2_test, predicted_counts_test_2)
print("R-squared:", r2)

# Calculate Mean Absolute Error (MAE) for additional insight
mae = mean_absolute_error(y_2_test, predicted_counts_test_2)
print("Mean Absolute Error:", mae)

0     39.218840
5     44.168787
16    45.297496
Name: mean, dtype: float64
            y
0   33.706647
5   41.019066
16  31.883199
R-squared: -3.710843122133163
Mean Absolute Error: 7.358737255579591


This is an alternate exploration from the problem with the negative binomial regression - with respect to the insufficient data of the train/test split. We can potentially consider a state-wide split. We don't need to structure our data from the bottom up, because we can take the average across FIPS codes.

In [ ]:
print(wastewater.shape)
print(store_hospital.shape)

(14586, 5)
(106490, 3)


In [ ]:
#aggregate by state -- first make state column


fips_to_state = {
    '01': 'Alabama', '02': 'Alaska', '04': 'Arizona', '05': 'Arkansas', '06': 'California',
    '08': 'Colorado', '09': 'Connecticut', '10': 'Delaware', '11': 'District of Columbia',
    '12': 'Florida', '13': 'Georgia', '15': 'Hawaii', '16': 'Idaho', '17': 'Illinois',
    '18': 'Indiana', '19': 'Iowa', '20': 'Kansas', '21': 'Kentucky', '22': 'Louisiana',
    '23': 'Maine', '24': 'Maryland', '25': 'Massachusetts', '26': 'Michigan', '27': 'Minnesota',
    '28': 'Mississippi', '29': 'Missouri', '30': 'Montana', '31': 'Nebraska', '32': 'Nevada',
    '33': 'New Hampshire', '34': 'New Jersey', '35': 'New Mexico', '36': 'New York',
    '37': 'North Carolina', '38': 'North Dakota', '39': 'Ohio', '40': 'Oklahoma', '41': 'Oregon',
    '42': 'Pennsylvania', '44': 'Rhode Island', '45': 'South Carolina', '46': 'South Dakota',
    '47': 'Tennessee', '48': 'Texas', '49': 'Utah', '50': 'Vermont', '51': 'Virginia',
    '53': 'Washington', '54': 'West Virginia', '55': 'Wisconsin', '56': 'Wyoming',
    '60': 'American Samoa', '66': 'Guam', '69': 'Northern Mariana Islands', '72': 'Puerto Rico',
    '78': 'Virgin Islands'
}

# Function to convert FIPS code to state
def fips_to_state_name(fips):
    state_fips = str(fips).zfill(5)[:2]  # Extract the first two digits and pad with zeros if needed
    return fips_to_state.get(state_fips, 'Unknown')  # Return 'Unknown' if the FIPS code is not found

# Apply the function to create the state column
wastewater['state'] = wastewater['fips'].apply(fips_to_state_name)
store_hospital['state'] = store_hospital['fips'].apply(fips_to_state_name)


print(wastewater)
print(store_hospital)
#adds one more column to each as expected



               fips          ds  percent_change_month  detect_prop_month  \
290388        26041  2021-11-01             -7.500000         100.000000   
106183        41067  2021-11-01            -23.885714         100.000000   
100604  29037,29095  2021-11-01             31.777778         100.000000   
394164        26077  2021-11-01              2.250000         100.000000   
213816        29043  2021-11-01            -25.000000         100.000000   
...             ...         ...                   ...                ...   
267367        26097  2024-06-01            -98.000000         100.000000   
264274        17015  2024-06-01            -69.000000          72.333333   
263165        50007  2024-06-01              1.714286         100.000000   
269782        27113  2024-06-01            -81.000000         100.000000   
504706        48097  2024-06-01             15.857143          59.571429   

        percentile_month      state  
290388         83.333500   Michigan  
106183     

In [ ]:


# Group by STATE and ds, then calculate the average of ptc_15d, detect_prop_15d, and percentile
average_values = wastewater.groupby(['state', 'ds']).agg({
    'percent_change_month': 'mean',
    'detect_prop_month': 'mean',
    'percentile_month': 'mean'
}).reset_index()


# Merge the averages back into the wastewater DataFrame
wastewater = pd.merge(wastewater, average_values, on=['state', 'ds'], how='left')
wastewater = wastewater.drop(columns=['fips'])



              fips          ds  percent_change_month_x  detect_prop_month_x  \
0            26041  2021-11-01               -7.500000           100.000000   
1            41067  2021-11-01              -23.885714           100.000000   
2      29037,29095  2021-11-01               31.777778           100.000000   
3            26077  2021-11-01                2.250000           100.000000   
4            29043  2021-11-01              -25.000000           100.000000   
...            ...         ...                     ...                  ...   
14581        26097  2024-06-01              -98.000000           100.000000   
14582        17015  2024-06-01              -69.000000            72.333333   
14583        50007  2024-06-01                1.714286           100.000000   
14584        27113  2024-06-01              -81.000000           100.000000   
14585        48097  2024-06-01               15.857143            59.571429   

       percentile_month_x      state  percent_chang

#TODO: FINISH LINEAR REGRESSION WITH ABOVE STATE-WIDE VIEW

#Gretel.AI

- Rows:


Health Related:
- Some objective metric of family disease?
- Food Security Levels
- BMI
- Hours of sleep a night

Work related:
- Distance from